# Test Scan merging

In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import os

from planeslam.geometry.util import quat_to_rot_mat
from planeslam.general import plot_3D_setup
import planeslam.io as io
from planeslam.extraction import pc_to_planes
from planeslam.scan import pc_to_scan

%load_ext autoreload
%autoreload 2

Read in airsim LiDAR and pose data

In [ ]:
# Read in point cloud data
binpath = os.path.join(os.getcwd(),'..', 'data', 'airsim', 'blocks_60_samples_loop_closure', 'lidar', 'Drone0')
PC_data = io.read_lidar_bin(binpath)

In [ ]:
# Read in ground-truth poses (in drone local frame)
posepath = os.path.join(os.getcwd(),'..', 'data', 'airsim', 'blocks_60_samples_loop_closure', 'poses', 'Drone0')
drone_positions, drone_orientations = io.read_poses(posepath)

Extract scans from LiDAR point clouds

In [ ]:
# Extract scans and planesets
num_scans = len(PC_data)
scans = num_scans * [None]
planesets = num_scans * [None]

for i in range(num_scans):
    scans[i] = pc_to_scan(PC_data[i])
    scans[i].transform(quat_to_rot_mat(drone_orientations[i,:]), drone_positions[i,:])

In [ ]:
# Plot scans
ax = plot_3D_setup(PC_data[0])

for scan in scans:
    scan.plot(ax, color='b')

Merge Scans

In [ ]:
merge = scans[0].merge(scans[1])

In [ ]:
# Plot merge
ax = plot_3D_setup(PC_data[0])
colors = ['r', 'b']

for i, scan in enumerate(scans[0:2]):
    scan.plot(ax, color=colors[i], show_normals=True)

merge.plot(ax, color='m', show_normals=True)

In [ ]:
merged = scans[0]

for s in scans[1:]:
    merged = merged.merge(s)

In [ ]:
# Plot merge
ax = plot_3D_setup(PC_data[0])
merged.plot(ax)